













































pip install jupyter-dash

! pip install plotly

In [1]:
import dash 
import dash_core_components as dcc 
import dash_html_components as html 
import dash_table
import plotly.express as px 
from dash.dependencies import Input, Output, State 
from dash.exceptions import PreventUpdate
import pandas as pd 
import os
# Imports 
%matplotlib inline
import matplotlib.pyplot as plt
import json                                                               
import numpy as np

import shutil
import sys


#install pyomo module if it's not already present in the libraries
if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

## Use the cbc solver (right now it is not working will look for another solver )
'''if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc")'''

import pyomo.environ as py

In [2]:
def weekly_orders():
    demand_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'demand_data.json')
    price_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'price_data.txt')
    qty_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'qty_data.txt')
    farms_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'farms.json')
    produce_path =  os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'produce.json')
    organic_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'organic_data.txt')
    local_data_path = os.path.join(r'C:\Users\Agnes\OneDrive\Documents\Internship\Python scripts', 'local_data.txt')


    with open(farms_path,"r") as f:
        farms = json.load(f)

    with open(produce_path,"r") as f:
        produce = json.load(f)

    with open(demand_data_path,"r") as f:
        demand_data = json.load(f)

    dic = ''
    with open(price_data_path,'r') as f:
             for i in f.readlines():
                dic=i #string
    price_data = eval(dic) # this is orignal dict with instace dict    

    dic = ''
    with open(qty_data_path,'r') as f:
             for i in f.readlines():
                dic=i #string
    qty_data = eval(dic) # this is orignal dict with instace dict

    dic = ''
    with open(organic_data_path,'r') as f:
             for i in f.readlines():
                dic=i #string
    organic_data = eval(dic) # this is orignal dict with instace dict

    dic = ''
    with open(local_data_path,'r') as f:
             for i in f.readlines():
                dic=i #string
    local_data = eval(dic) # this is orignal dict with instace dict

    ## Creating our Model: We would be using the Abstract model. We would define all the different blocks of our function. Then afterwards we would import the data.

    model = py.ConcreteModel()

    ## Define the different sets 
    model.PRODUCE = py.Set(initialize = produce)     # set of all produce 
    model.FARMS =   py.Set(initialize = farms)   # set of all the farms  
    model.local =   py.Set(initialize = local_data)
    model.organic = py.Set(model.PRODUCE,model.FARMS,initialize = organic_data)    # set of all the organic farms '''

    ## Define Parameters
    #model.c = py.Param(within = NonNegativeReals,data = initialize())
    model.p = py.Param(model.PRODUCE,model.FARMS,initialize = price_data)
    model.a = py.Param(model.PRODUCE,model.FARMS, initialize = qty_data)
    model.b = py.Param(model.PRODUCE,initialize = demand_data)
    model.alpha = py.Param(initialize = .85)
    model.beta = py.Param(initialize = 1.0)


    221## Declare decision variables 
    model.x = py.Var( model.PRODUCE*model.FARMS, domain = py.NonNegativeReals,initialize = 0)#qty of produce of type i bought from farm j 
    model.s = py.Var(model.PRODUCE, model.FARMS, domain = py.Binary,initialize = 1)

    ## Define constraints

    # Demand constraints 
    def demand_constraint(model,i):
        return sum(model.x[i,j] for j in model.FARMS) == model.b[i] 
    model.demand = py.Constraint (model.PRODUCE, rule = demand_constraint)

    # Availability constraints 
    def availability_constraint(model,i,j):
        return (model.x[i,j] - model.a[i,j]*model.s[i,j] <= 0 )
    model.available = py.Constraint(model.PRODUCE, model.FARMS, rule = availability_constraint)
    #model.available = py.Constraint(model.PRODUCE, model.FARMS, rule = lambda model,i,j : model.x[i,j] <=  model.a[i,j])


    ## Organic constraints:
    def org_constraint(model):
        return (sum(model.x[i,j] for (i,j) in model.organic) >= sum(model.x[i,j] for i in model.PRODUCE for j in model.FARMS)*model.beta)
    model.org = py.Constraint(rule = org_constraint)

    ## Local constraints
    def local_constraint(model):
         return (sum(model.x[i,j] for (i,j) in model.local) >= sum(model.x[i,j] for i in model.PRODUCE for j in model.FARMS)*model.alpha)
    model.loc = py.Constraint(rule = local_constraint)

    ## Declare objective 
    def obj_expression(model):
        return ( sum(model.p[i, j] * model.x[i, j] for i in model.PRODUCE for j in model.FARMS))
    model.profit = py.Objective(rule = obj_expression)

    # unique constraints 
    def unique_constraint(model, i):
        return (sum(model.s[i,j] for j in model.FARMS )== 1)
    model.unique = py.Constraint(model.PRODUCE, rule = unique_constraint)

    ## Local constraints 
    def loc_constraint(model):
        return (sum(model.x[i,j] for i in model.organic) <= model.x[i,j]*model.alpha)

    solver = py.SolverFactory('glpk')
    result = solver.solve(model)
    
    optimal_values = [(key,py.value(model.x[key])) for key in model.x]
    df = pd.DataFrame(optimal_values)
    df = df.reset_index()
    df['Produce'] = df.loc[:,0].apply(lambda x: x[0])
    df['Farm']  = df.loc[:,0].apply(lambda x: x[1])
    df = df[['Produce','Farm',1]]
    df = df.rename(columns= {1 : 'Qty'})
    df = df[df['Qty'] != 0]
    return df


In [3]:
df = weekly_orders()
df

,Produce,Farm,Qty
1,"Allium, Green Onion",FarmToTable,55.0
5,Beets,FarmToTable,105.0
8,Carrots,JBG,150.0
14,Collard Greens,Farmshare,22.0
16,"Kale, Lacinato",JBG,18.0
20,"Radish, Average Variety",JBG,16.0
26,Spinach,Farmshare,18.0
29,"Squash, Summer",FarmToTable,20.0
34,Swiss Chard,Farmshare,58.0


In [ ]:
df1 = pd.read_excel(r'C:\Users\Agnes\OneDrive\Documents\Internship/synthetic_data.xlsx',
                  sheet_name = 'Demand')

df2 = pd.read_excel(r'C:\Users\Agnes\OneDrive\Documents\Internship/synthetic_data.xlsx',
                  sheet_name = 'Available')
df = weekly_orders()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

colors = {
    "graphBackground": "#F5F5F5",
    "background": "#ffffff",
    "text": "#000000"
    }

app.layout = html.Div([
    html.H1(children = 'Farmshare Dashboard'),
    dcc.Location(id ='url', refresh=False),
    html.Div(id ='menu-content')
        ])

index_page = html.Div([
        dcc.Link('Orders', href='/page-1'),
        html.Br(),
        dcc.Link('Data Modeling', href='/page-2'),
   ])

page_1_layout = html.Div([    
    html.H1('Orders'), 
    dash_table.DataTable(
        id = 'adding-rows-table',
        columns = [ {"name": i, "id": i, "deletable": True, "selectable": True} for i in df1.columns],
        data = df1.to_dict('records'),
        editable = True,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = "multi",
        row_selectable ="multi",
        row_deletable = True,
        selected_columns = [],
        selected_rows = [],
        export_format='xlsx',
        export_headers='display',
    ),

    html.Button("Add row", id = 'editing-rows-button', n_clicks = 0),

    html.H1('Produce availability'), 
    dash_table.DataTable(
            id = 'available-table',
            columns = [ {"name": i, "id": i, "deletable": True, "selectable": True} for i in df2.columns],
            data = df2.to_dict('records'),
            editable = True,
            row_deletable = True,
            export_format='xlsx',
            export_headers='display',
            merge_duplicate_headers=True,
        ),  
        html.Button("Add row", id = 'editing-rows-button-2', n_clicks = 0),
        html.Br(),
        dcc.Link('Data Modeling', href='/page-2'),
        html.Br(),
        dcc.Link('Go back to home', href='/'),    
])

@app.callback(
    Output('adding-rows-table', 'data'),
    Input('editing-rows-button', 'n_clicks'),
    State('adding-rows-table', 'data'),
    State('adding-rows-table', 'columns'))

def add_row(n_clicks, rows, columns):
    if n_clicks > 0:
         rows.append({c['id']: '' for c in columns})
    return rows 

@app.callback(
    Output('available-table', 'data'),
    Input('editing-rows-button-2','n_clicks'),
    State('available-table', 'data'),
    State('available-table', 'columns'))

def add_row(n_clicks, rows, columns):
    if n_clicks > 0:
         rows.append({c['id']: '' for c in columns})
    return rows    

page_2_layout = html.Div([
    html.H1('Data Modeling'),
    html.Br(),
    
    html.Button("Run model and display results", id = 'run-model-button', n_clicks = 0),   
    
    html.Div(id="div-1"),
 
    html.Br(),
    dcc.Link('Orders', href='/page-1'),
    html.Br(),
    dcc.Link('Go back to home', href='/'),
    
])

@app.callback (
    Output('div-1', 'children'),
    Input('run-model-button','n_clicks'),
)
def update_result_table(n_clicks):   
    if n_clicks:
        df= weekly_orders()
        #print(df)
        return html.Div([dash_table.DataTable(
                                id = 'result-table',
                                data = df.to_dict('records'), 
                                columns =   [{"name": i, "id": i,} for i in (df.columns)],
                                export_format = 'xlsx',
                                export_headers = 'display')])
               
# Update the index
@app.callback(dash.dependencies.Output('menu-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])

def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page
    # You could also return a 404 "URL not found" page here

    
if __name__ == '__main__':
    app.run_server(debug=False, port =3004)

Dash is running on http://127.0.0.1:3004/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3004/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jul/2021 08:16:44] "GET /page-1 HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:16:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:16:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:16:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:16:45] "GET /_favicon.ico?v=1.21.0 HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:16:45] "GET /_dash-component-suites/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:16:45] "GET /_dash-component-suites/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:16:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:16:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:24:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 08:24:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.

'''app.callback (
    Output('result-table', 'data'), 
    Output('result-table', 'columns'),
    Input('run-model-button','n_clicks')
    )
def update_result_table(n_clicks):   
    if n_clicks>=1:
        df1 = weekly_orders()
        #data =df1.to_dict("records")
        #columns = [{"name": i, "id": i} for i in df1.columns]
        #print(data)
        #print(columns)
        df = pd.read_excel(r'C:\Users\Agnes\OneDrive\Documents\Internship/synthetic_data.xlsx',
                  sheet_name = 'Demand')
        columns = [{"name": i, "id": i} for i in df.columns]
        data = df.to_dict('records')
        return dash_table.DataTable(data=df_data, columns=columns)

    else:
        return ({"display":"None"},{"display":"None"})
'''

   ''' dash_table.DataTable (
        id = 'result-table',
        columns= [],
        data=[],
        export_format='xlsx',
        export_headers='display',
    ),'''

In [ ]:
+